# Everything about PCA in VAMPIRE Analysis

27 March 2022

Objective: Explore the purpose of PCA in VAMPIRE analysis.

Hypothesis: PCA acts as a noise reduction tool and a dimensionality reduction tool.

## Baseline data setup

### Baseline VAMPIRE analysis

Baseline VAMPIRE analysis applies PCA on normalized contours, followed by K-Means clustering of principal components in the PC space.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import vampire as vp
import umap
from sklearn import manifold
vp.plot.set_plot_style()

In [ ]:
img_set_path = r'C:\Files\github-projects\nance-lab-data\microfiber\ogd_severity_segmentations'
output_path = r'C:\Files\github-projects\nance-lab-data\microfiber\result\result-2022-03-27'

In [ ]:
# extract properties from all images for future use
vp.extraction.extract_properties(img_set_path)

In [ ]:
build_info_df = pd.DataFrame({
        'img_set_path': [img_set_path],
        'output_path': [output_path],
        'model_name': ['otsu'],
        'num_points': [np.nan],
        'num_clusters': [np.nan],
        'num_pc': [np.nan],
        'threshold': ['otsu'],
    })
vp.quickstart.build_models(build_info_df, random_state=1)

In [ ]:
model_path = os.path.join(output_path, 'model_otsu_(50_5_32)__otsu.pickle')
vampire_model = vp.util.read_pickle(model_path)

In [ ]:
apply_info_df = pd.DataFrame({
        'img_set_path': [img_set_path],
        'model_path': [model_path],
        'output_path': [output_path],
        'img_set_name': ['otsu'],
    })
vp.quickstart.apply_models(apply_info_df)

In [ ]:
property_path = os.path.join(output_path, 'apply-properties_otsu_on_otsu_(50_5_32)__.pickle')
apply_properties_df = vp.util.read_pickle(property_path)

### Data processing

#### Label experimental metadata

In [ ]:
apply_properties_df.head()

In [ ]:
def label_df(properties_df, id_df, target_props, search_prop='filename'):
    """
    properties_df : DataFrame
        Contains "filename" column
    id_df : DataFrame
        Contains columns listed in `properties` and "regex" column.
    target_props : list
        List of property(s) to be added. Must match column name in `id_df`.
    """
    properties_df = properties_df.copy()
    properties_df[target_props] = np.nan
    for i in range(id_df['regex'].size):
        mask = properties_df[search_prop].astype(str).str.contains(id_df['regex'][i])
        properties_df.loc[mask, target_props] = id_df.iloc[i, :][target_props].values
    return properties_df

In [ ]:
slice_id_df = pd.read_excel(r'C:\Files\github-projects\nance-lab-data\microfiber\slice-labels.xlsx')
slice_id_df['regex'] = slice_id_df['slice_id'] + '_'
slice_id_df.head()

In [ ]:
region_names = ['cortex', 'thalamus', 'hippocampus']
region_id_df = pd.DataFrame({
    'regex': region_names,
    'region': region_names,
})

In [ ]:
apply_properties_df = label_df(apply_properties_df, slice_id_df, ['slice_id', 'treatment'])
apply_properties_df = label_df(apply_properties_df, region_id_df, ['region'])

In [ ]:
apply_properties_df

#### Label coloring

In [ ]:
from matplotlib.colors import to_hex
color_id_df = pd.DataFrame({
    'cluster_id': [0, 1, 2, 3, 4],
    'color': [to_hex(plt.get_cmap('twilight')(np.linspace(0.1, 0.9, 5))[i]) for i in range(5)],
})
color_id_df['regex'] = color_id_df['cluster_id'].astype(str)

In [ ]:
label_colors = label_df(apply_properties_df, color_id_df, ['color'], 'cluster_id').color.values
label_colors

## Contour visual representations (This should be moved to KMeans clustering)

The paper states that contours are represented by centroid reconstruction of each cluster. However, the mean contour of each cluster is used instead. Here, we comapre the two types of visualization.

- Cluster centroid reconstruction (Paper)
- Mean contour of each cluster (Implementation)

### Cluster centroid reconstruction

To reconstruct the shape, we use 

$$\text{PCA reconstruction = (PC score)(principal directions) + mean}$$

[Additional resource: How to reverse PCA and reconstruct original variables from several principal components?](https://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-comhttps://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-comhttps://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-comhttps://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-com)

In [ ]:
fig, axs = plt.subplots(1, vampire_model.num_clusters, figsize=(10, 2))
principal_directions = vampire_model.principal_directions[:, :vampire_model.num_pc].T
mean_contour = vampire_model.mean_aligned_contour
for i in range(vampire_model.num_clusters):
    # calculate reconstructions
    centroid_pc = vampire_model.centroids[i]
    centroid_coord = centroid_pc @ principal_directions + mean_contour
    centroid_x = centroid_coord[:vampire_model.num_points]
    centroid_y = centroid_coord[vampire_model.num_points:]
    # plot reconstructions
    axs[i].plot(centroid_x, centroid_y)
    axs[i].axis('equal')
    axs[i].axis('off')

### Mean contour of each cluster

In [ ]:
vp.plot.plot_contours(vampire_model)

### Discussion

Comparing the two methods, the cluster centroid reconstruction has less variation between groups than mean contour of each cluster. Small change from centroid reconstruction is a large change in the real contour space.

We can plot some contours randomly selected representatives from each cluster as our standard for comparison. The mean contour of each cluster resembles the representatives the most, as expected, because overlay of the representatives are the mean representation of the contours.

(I would call the centroid reconstruction as "shape mode" and the mean contour of each cluster as "mean representation of shape mode", but the original paper calls the mean contour as "shape mode".)

In [ ]:
vp.plot.plot_representatives(vampire_model, apply_properties_df, num_sample=50, random_state=1, alpha=0.2)

**Conclusion**: We will stick with mean contour of each cluster for visualization purposes. It's more easy to identify and associate features to them.

## Baseline PCA analysis

### PCA explained variance

In [ ]:
vampire_model.num_pc

In [ ]:
plt.plot(vampire_model.explained_variance_ratio, 'o-')
plt.plot(vampire_model.num_pc, 
         vampire_model.explained_variance_ratio[vampire_model.num_pc], 'ro-')
plt.xlabel('PC number')
plt.ylabel('Explained variance ratio')

^ The explained variance ratio of each PC decreases dramatically at first two. Elbow rule suggests that we can keep the first 2, 5, or 9 PCs, depending on our judgement.

In [ ]:
plt.plot(vampire_model.cum_explained_variance_ratio, 'o-')
plt.plot(vampire_model.num_pc, 
         vampire_model.cum_explained_variance_ratio[vampire_model.num_pc], 'ro-')
plt.xlabel('Number of PC')
plt.ylabel('Cumulative explained variance ratio')

^ The cumulative explained variance ratio at 2, 5, and 9 is ~40%, 65%, and 75%. To cover 95% of total variance, the first 32 PCs are kept (shown in red).

The plots suggests that PCA filters and denoises the data by neglecting non-contributing portion in the PC space.

### PC space visualization

In [ ]:
Y = vp.analysis.pca_transform_contours(normalized_contours,
                                       vampire_model.mean_aligned_contour,
                                       vampire_model.principal_directions)

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1], s=1)
plt.xlabel(f'PC 1 ({round(vampire_model.explained_variance_ratio[0]*100)}%)')
plt.ylabel(f'PC 2 ({round(vampire_model.explained_variance_ratio[1]*100)}%)')

^ The first two PC captures ~42% total variance. There is no clear separation between clusters. Rather, we see a big cluster.

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1], s=1, color=label_colors)
plt.xlabel(f'PC 1 ({round(vampire_model.explained_variance_ratio[0]*100)}%)')
plt.ylabel(f'PC 2 ({round(vampire_model.explained_variance_ratio[1]*100)}%)')

^ After annotating with result of VAMPIRE analysis cluster, we see that the first two PC separates all clusters expect the dark cluster (cluster 3). 

```{hint}
This may suggest that four clusters is enough to capture the shape variations. However, this may also be cautioned that only less than 50% of total variance is captured.
```

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1], s=1, color=label_colors, alpha=0.1)
plt.xlabel(f'PC 1 ({round(vampire_model.explained_variance_ratio[0]*100)}%)')
plt.ylabel(f'PC 2 ({round(vampire_model.explained_variance_ratio[1]*100)}%)')

^ Changing alpha helps to gauge numbers of samples in each cluster.

In [ ]:
plt.scatter(Y[:, 0], Y[:, 2], s=1, color=label_colors, alpha=0.5)
plt.xlabel(f'PC 1 ({round(vampire_model.explained_variance_ratio[0]*100)}%)')
plt.ylabel(f'PC 3 ({round(vampire_model.explained_variance_ratio[2]*100)}%)')

In [ ]:
plt.scatter(Y[:, 1], Y[:, 2], s=1, color=label_colors, alpha=0.5)
plt.ylabel(f'PC 2 ({round(vampire_model.explained_variance_ratio[1]*100)}%)')
plt.xlabel(f'PC 3 ({round(vampire_model.explained_variance_ratio[2]*100)}%)')

^ Plots of PC 2 and 3 also do not show effective separation of clusters.

## Elimination of PCA for direct clustering

In [ ]:
normalized_contours = np.vstack(apply_properties_df['normalized_contour'].values)

In [ ]:
direct_cluster_id_df, direct_centroids, inertia = vp.analysis.cluster_contours(normalized_contours, num_clusters=5, num_pc=100)

In [ ]:
direct_cluster_id_df.head()

In [ ]:
direct_df = pd.concat([apply_properties_df.drop(['cluster_id', 'distance_to_centroid'], axis=1), direct_cluster_id_df], axis=1)

In [ ]:
direct_df.head()

### Cluster centroid

In [ ]:
fig, axs = plt.subplots(1, vampire_model.num_clusters, figsize=(10, 2))
for i in range(vampire_model.num_clusters):
    centroid_x = direct_centroids[i, :vampire_model.num_points]
    centroid_y = direct_centroids[i, vampire_model.num_points:]
    # plot reconstructions
    axs[i].plot(centroid_x, centroid_y)
    axs[i].axis('equal')
    axs[i].axis('off')

### Mean contour of each cluster

In [ ]:
fig, axs = plt.subplots(1, vampire_model.num_clusters, figsize=(10, 2))
for i in range(vampire_model.num_clusters):
    cluster_mask = direct_df['cluster_id'] == i
    normalized_contour = np.vstack(direct_df[cluster_mask]['normalized_contour'])
    mean_normalized_contour = normalized_contour.mean(axis=0)
    contour_x = mean_normalized_contour[:vampire_model.num_points]
    contour_y = mean_normalized_contour[vampire_model.num_points:]
    axs[i].plot(contour_x, contour_y)
    axs[i].axis('equal')
    axs[i].axis('off')

In [ ]:
vp.plot.plot_contours(vampire_model)

In [ ]:
Y = vp.analysis.pca_transform_contours(normalized_contours,
                                       vampire_model.mean_aligned_contour,
                                       vampire_model.principal_directions)

In [ ]:
Y.shape

## PCA on morphological parameters

- https://stats.stackexchange.com/questions/183236/what-is-the-relation-between-k-means-clustering-and-pca
- https://pair-code.github.io/understanding-umap/